In [ ]:
import numpy as np # linear algebra
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict

**Load Data**

In [ ]:
path = "../input/netflix-shows/netflix_titles.csv"
df = pd.read_csv(path)
df.head()

**Checking for null values**

In [ ]:
df.isnull().sum()

Уберем ненужные столбцы

In [ ]:
df = df.drop('show_id', axis=1)

****

# **EDA**

**Распределение по жанру**

In [ ]:
sns.countplot(data=df, x='type')

**Колчество выпущенного контента в зависимости от года выпуска**

In [ ]:
plt.figure(figsize=(4, 4), dpi=200)
sns.countplot(data=df, y='release_year', order=df['release_year'].value_counts().index[0:20])

**Страны с самым большим количеством выпущенного контента**

In [ ]:
df['country']

Видно, что есть проблема с тем, что у некоторых фильмов больше чем 1 страна производства, поэтому создадим Так же в этом столбце есть пропуски.

In [ ]:
#Заполним пропуски
df['country'] = df['country'].fillna('unknown')

In [ ]:
countries = defaultdict(int)
for l in list(df['country'].apply(lambda x: x.split(', '))):
    for item in l:
        countries[item] +=1

countries_sorted = {k:v for k,v in sorted(countries.items(), key = lambda x: x[1],reverse=True)}

In [ ]:
plt.figure(figsize=(4, 4), dpi=200)
sns.barplot(y=list(countries_sorted.keys())[:20], x=list(countries_sorted.values())[:20])

***В какое время года на нетфликс загружается больше фильмов?***

In [ ]:
df['date_added']

In [ ]:
number_list = np.arange(1, 13)
month_list = ['Январь', 'Февраль', 'Март', 'Апрель', 'Май', 'Июнь', 'Июль', 'Август', 'Сентябрь', 'Октябрь', 'Ноябрь', 'Декабрь']
month_map = {k:v for k,v in zip(number_list, month_list)}

In [ ]:
df = df.dropna(subset=['date_added'])

In [ ]:
# Создадим новый признак
df['month_added'] = pd.to_datetime(df['date_added']).apply(lambda x: int(x.month)).map(month_map)

In [ ]:
plt.figure(figsize=(4, 4), dpi=200)
sns.countplot(data=df, y='month_added', order = month_list)

Видно, что в летние/весенние месяцы контента выходит меньше, чем осенью и особенно зимой, так как скорее всего это более выгодно делать зимой/осенью, потому что люди проводят больше времени дома.

**Самый популярный главный актер в американских фильмах**

In [ ]:
is_USA = df['country'].apply(lambda x: 'United States' in x)

In [ ]:
df_USA = df[is_USA].dropna(subset=['cast'])

In [ ]:
df_USA['lead_actor'] = df_USA['cast'].apply(lambda x: x.split(', ')[0])

In [ ]:
df_USA[df_USA['lead_actor'] == 'Adam Sandler']['title']

In [ ]:
sns.countplot(data=df_USA, y=df_USA['lead_actor'], order=df_USA['lead_actor'].value_counts().index[0:20])

***Самые высоко оценненные фильмы/сериалы c их жанрами***

Рейтинги возьмем из таблицы imdb-extensive-dataset 

In [ ]:
imdb_movies = pd.read_csv('/kaggle/input/imdb-extensive-dataset/IMDb movies.csv')

In [ ]:
ratings = pd.DataFrame({'title': imdb_movies['title'], 'avg_vote': imdb_movies['avg_vote']})
ratings.drop_duplicates(subset=['title','avg_vote'], inplace=True)

In [ ]:
#Соединим таблицы
df_ratings = ratings.merge(df.copy(), left_on='title', right_on='title', how='inner')
df_ratings = df_ratings.sort_values(by='avg_vote', ascending=False)

In [ ]:
import plotly.express as px
plt.figure(figsize=(10,10))
top_rated=df_ratings[0:10]
fig =px.sunburst(
    top_rated,
    path=['title','listed_in'],
    values='avg_vote',
    color='avg_vote',
)
fig.show()